In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import mlflow
import mlflow.sklearn


mlflow.set_tracking_uri('http://54.195.173.154:5000/')
mlflow.set_experiment('mlflow_dvc')

X = pd.read_csv('features.csv', usecols=['f1','f2','f3','f4'])
Y = pd.read_csv('label.csv', usecols=['label'])

with mlflow.start_run(run_name="experiment") as run: 
    # tracking run parameters
    mlflow.log_param("compute", 'local')
    mlflow.log_param("dataset", 'example')
    mlflow.log_param("dataset_version", '2.0')
    mlflow.log_param("dataset_path", 's3://score-journey-boluo/data/')
    mlflow.log_param("algo", 'random forest example')
    
    # tracking any additional hyperparameters for reproducibility
    n_estimators = 5
    mlflow.log_param("n_estimators", n_estimators)

    # train the model
    rf = RandomForestRegressor(n_estimators=n_estimators)
    rf.fit(X, Y)
    Y_pred = rf.predict(X)

    # automatically save the model artifact to the S3 bucket for later deployment
    mlflow.sklearn.log_model(rf, "rf-baseline-model")

    # log model performance using any metric
    mse = mean_squared_error(Y, Y_pred)
    mlflow.log_metric("mse", mse)
    
    mlflow.end_run()

2022/11/25 15:55:47 INFO mlflow.tracking.fluent: Experiment with name 'mlflow_dvc' does not exist. Creating a new experiment.
/tmp/ipykernel_31939/29304171.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X, Y)
/home/ubuntu/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
!pip install mlflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 58.7 MB/s eta 0:00:0000:0100:01
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 38.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 48.5 MB/s eta 0:00:00
  Using cached sqlparse-0.4.3-py3-none-any.whl (42 kB)
  Using cached docker-6.0.1-py3-none-any.whl (147 kB)
  Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
  Using cached protobuf-4.21.9-cp37-abi3-manylinux2014_x86_64.whl (408 kB)
  Using cached alembic-1.8.1-py3-none-any.whl (209 kB)
  Using cached databricks-cli-0.17.3.tar.gz (77 kB)
  Preparing metadata (setup.py) ... done
  Using cached Mako-1.2.4-py3-none-any.whl (78 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Created wheel for databricks-cli: filename=databricks_cli-0.17.3-py3-none-any.whl size=139084 sha256=eb632f641f3e24fc57d5c5ccf0f1e8dc0e43267c828f34288bf1623398eeb997
  St